![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx and `LangChain` to make a series of calls to a language model

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate Simple Sequential Chain using langchain integration with Watsonx models. There are 2 sections:
1. Using a simple sequential chain with 2 different LLMs to perform 2 generation tasks.
2. Using a simple sequential chain to perform an extraction task followed by a classification task using the a single LLM.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to chain `google/flan-ul2` and `google/flan-t5-xxl` models to generate a sequence of chains. You generate a random question on a given topic and then generate an answer to that question. You utilize the LangChain framework, using a simple chain (LLMChain) and the extended chain (SimpleSequentialChain) with WatsonxLLM.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models in Watsonx.ai](#models)
- [WatsonxLLM interface](#watsonxllm)
- [Simple Sequential Chain experiment](#experiment)
- [Summarize customer complaints](#summarize)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a Watson Studio Project and associate a Watson Machine Learning service to the project.
-  Download the LangChain Integration.ipynb Python Notebook to your local computer so that you can upload it to Watson Studio.  

### Install and import the `datasets` and dependecies

In [ ]:
# Installing the Watson Machine Learning, Pydantic and LangChain packages

!pip install "ibm-watson-machine-learning>=1.0.320" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install langchain | tail -n 1

**Note:** It is good practice and recommended to install all the required Python packages at the beginning of the notebook.
However, to show what packages and libraries are required to execute a block of code successfully, the installation
and importing of spcecific libraies were moved down to the cells where the library is called. 
This was done for educational purposes and mainly for two reasons:
- To highlight the libraries that are needed and when they are needed
- To make the connection between the library and the code

There is also a print statement at the end of most cells. If you want to see the output of a cell, uncomment the print statement and rerun the code in the cell.

### Defining the WML credentials
IBM watsonx.ai works with Large Langauge Models (LLMs) in the context of a project, and uses the resources and services available to the project. In the code block below, we will first attempt to retrieve the project ID from where this notebook is running. If there is an error, then you will need to provide the project ID explicitly.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [ ]:
# Input your WML API key when requested when you run the cell
# It is good practice not no hard-code you credentials in a notebook, but let the user provide it during runtime

import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

url = credentials['url']
apikey = credentials['apikey']

#print(credentials)

### Defining the project id
IBM watsonx.ai works with Large Langauge Models (LLMs) in the context of a project, and uses the resources and services available to the project. In the code block below, we will first attempt to retrieve the project ID from where this notebook is running. If there is an error, then you will need to provide the project ID explicitly.

In [ ]:
# Import the os library and retrieve the project ID from the underlying operation system environment

import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

#print(project_id)

<a id="models"></a>
## Foundation Models in `watsonx.ai`

#### List available models

All avaliable models are presented under `ModelTypes` class.

In [ ]:
# Import a required WML library and list the foundation models currently hosted on the watsonx.ai platform  

from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

You need to specify `model_id`s to identify the LLMs you are using. In this notebook, the `flan-ul2` and `flan-t5-xxl` models are used (as shown below). You can experiement with other models. Review the cell below for correct syntax (all lowecase including company that created and trained the LLM).

In [ ]:
# Create model ids for each model that will be used in the rest of the notebook

model_id_1 = "google/flan-ul2"
model_id_2 = "google/flan-t5-xxl"

#print(model_id_1)
#print(model_id_2)

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class. Note that in this part of the lab, you are using the same parameters for both the `flan-ul2` and the `flan-t5-xxl` models. You can choose to use different parameters (this will be demonstrated in the second part of this notebook below).

**Action:** If any complications please refer to the [documentation](https://ibm.github.io/watson-machine-learning-sdk/).

In [ ]:
# Import the required WML libraries

from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

# Specify the LLM model parameters

parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 100,
    "min_new_tokens": 1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 1
}

<a id="watsonxllm"></a>
## WatsonxLLM interface

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `CustomLLM` check the [LangChain documentation](https://python.langchain.com/docs/integrations/llms/ibm_watsonx/)

In [ ]:
# Install the langchain_ibm package that facilitates LangChain integration with IBM watsonx.ai models
# From langchain_ibm import WatsonxLLM

!pip install -U langchain-ibm
from langchain_ibm import WatsonxLLM

# WatsonxLLM puts a wrapper around the foundation models to make it compatible with LangChain

flan_ul2_llm = WatsonxLLM(
    model_id=model_id_1,
    url=url,
    project_id=project_id,
    params=parameters,
    apikey=apikey)

flan_t5_llm = WatsonxLLM(
    model_id=model_id_2,
    url=url,
    project_id=project_id,
    params=parameters,
    apikey=apikey)

You can print all set data about the WatsonxLLM object using the `dict()` method. Here you are only printing it for the `flan-ul2` model. The `flan-t5-xxl` model uses the same set of parameters. 

In [ ]:
# Print the parameters associated with the WatsonLLM object (LLM model)

flan_ul2_llm.dict()

<a id="experiment"></a>
## Simple Sequential Chain experiment

The simplest type of sequential chain is called a `SimpleSequentialChain`, in which each step has a single input and output and the output of one step serves as the input for the following step.

The experiment consists of generating a random question about any topic with one LLM (`flan-ul2`) and then answering the question with the other LLM (`flan-t5-xxl`).

An object called `PromptTemplate` assists in generating prompts using a combination of user input, additional non-static data, and a fixed template string.

In our case we would like to create two `PromptTemplate` objects which will be responsible for creating a random question and answering it.

In [ ]:
# Import the required langchain library

from langchain.prompts import PromptTemplate

# Create two prompts
# The 1st prompt contains the structure of the 1st Question with a dynamic topic provided  by the user. This is sent to flan-ul2 to generate a completion. 
# The 2nd prompt contains the structure of the 2nd Question with dynamic question (generated by flan-ul2). This is sent to flan-t5-xxl to generate a final answer.  

template_1 = "Generate a random question about {topic}: Question: "
prompt_1 = PromptTemplate.from_template(template_1)

template_2 ="Answer the following question: {question}"
prompt_2 = PromptTemplate.from_template(template_2)

#print(prompt_1)
#print(prompt_2)

We would like to add functionality around language models using `LLMChain` chain (deprecated). The deprecated syntax as well as the new 
and alternative syntaxes on how to construct a chain (also called a runnable sequence) ia given in the cells below. The new syntax uses 
LCEL (LangChain expression language), a declarative way to easily compose chains together. The core functionality however did not change. 

The `prompt_to_flan_ul2` chain formats the prompt template whose task is to take the user input and then pass the formatted string to the first LLM (`flan-ul2`) and then returns the LLM's output.

In [ ]:
# Import the required LangChain library

#from langchain.chains import LLMChain
from langchain_core.runnables import RunnableSequence

# Create a simple chain consisting of the 1st prompt and the 1st LLM

#prompt_to_flan_ul2 = LLMChain(prompt=prompt_1, llm=flan_ul2_llm) - deprecated syntax
#prompt_to_flan_ul2 = prompt_1 | flan_ul2_llm - alternative new syntax
prompt_to_flan_ul2 = RunnableSequence(first=prompt_1, last=flan_ul2_llm)

#print(prompt_to_flan_ul2)

The `ul2_to_t5` chain formats the prompt template whose task is the take the question generated by the `prompt_to_flan_ul2` chain, passes the formated string to the 2nd LLM (`flan-t5-xxl`), and returns its output.

In [ ]:
# Create a 2nd simple chain consisting of the 2nd prompt and the 2nd LLM 

#ul2_to_t5 = LLMChain(prompt=prompt_2, llm=flan_t5_llm) - deprecated syntax
#ul2_to_t5 = prompt_2 | flan_t5_llm - alternative new syntax
ul2_to_t5 = RunnableSequence(first=prompt_2, last=flan_t5_llm)

#print(ul2_to_t5)

This is the overall chain where we run `prompt_to_flan_ul2` and `ul2_to_t5` chains in sequence.

Generate random question and answer to topic.

In [ ]:
# Import the callback handler that prints chain components' inputs and outputs to the console
# This is very helpful to verify that the chain is working as intended

from langchain.callbacks.tracers import ConsoleCallbackHandler

In [ ]:
# Providing the chained output. You pass in the word "Australia" and prompt_to_flan_ul2 will generate a random question on Australia. 
# This question is then passed to ul2_to_t5 to get an answer to that random question.

qa = RunnableSequence(first=prompt_to_flan_ul2, last=ul2_to_t5)

chain_output = qa.invoke("Australia", config={'callbacks': [ConsoleCallbackHandler()]})

#print(chain_output)

The example above is a simple sequential chain where the output of one LLM is passed as the input to a second LLM. Note that there is no absolute need for 2 different LLMs. The example below shows a more complex sequnetial chain where the same LLM (flan-ul2 in this example) is used for 2 different use cases.

1. In the first part of the chain, the flan-ul2 model is used to extract information from the input.
2. In the second part of the chain, the extracted information is passed to flan-ul2 again to perform a classisifcation task.

In addition, you will use different parameters setting for the first extraction task (with sampling decoding and a max_new_tokens setting of 100) from the second classification task (this uses greedy decoding with a max_new_tokes of 10 only). In both cases, you are using the flan-ul2 model. 

<a id="summarize"></a>
## Summarize and classify customer complaints chain

In [ ]:
# We are reusing variables that were declared earlier, but we're creating different model types
# In the example FLAN_U2 is used in both instances, but the models may be different
# We have different sets of parameters for each model as the yes or no classification needs very few tokens

model_id_1 = "google/flan-ul2"
model_id_2 = "google/flan-ul2" 

model1_parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 100,
    "min_new_tokens": 1,
    "top_k": 50,
    "top_p": 1    
}

model2_parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 10,
    "min_new_tokens": 1,
    "top_k": 50,
    "top_p": 1       
}

As you have done in the earlier example, you use WatsonxLLM to provide a wrapper around foundation models. Note that the name clearly identifies the purpose of each wrapper.

In [ ]:
# WatsonxLLM puts a wrapper around the foundation models to make it compatible with LangChain

flan_ul2_llm_extract = WatsonxLLM(
    model_id=model_id_1,
    url=url,
    project_id=project_id,
    params=parameters,
    apikey=apikey)

flan_ul2_llm_classify = WatsonxLLM(
    model_id=model_id_2,
    url=url,
    project_id=project_id,
    params=parameters,
    apikey=apikey)

In [ ]:
# Use for testing/debugging
# flan_ul2_llm_extract.dict()
# flan_ul2_llm_classify.dict()

The first part of the chain asks `flan-ul2` to analyze the user-provided prompt text to extract 3 complaints. The second part of the chain again uses `flan-ul2` to classify the 3 complaints and classify them to see if the complaints involve identity theft.

In [ ]:
# Create prompt templates for summarizing and classifying customer complaints

template_1 = "From the following customer complaint, extract 3 factors that caused the customer to be unhappy. Put each factor on a new line. Customer complaint: {customer_complaint}. Numbered list of complaints: 1."
prompt_1 = PromptTemplate.from_template(template_1)

template_2 ="Does the following statements contain the concept of identify theft?: {list_of_complaints}"
prompt_2 = PromptTemplate.from_template(template_2)

#print(prompt_1)
#print(prompt_2)

In [ ]:
# Create the LLM chains: model + prompt

#get_list_of_complaints = LLMChain(llm=flan_ul2_llm_extract, prompt=prompt_1) - deprecated syntax
get_list_of_complaints = RunnableSequence(first=prompt_1, last=flan_ul2_llm_extract)
#check_for_identify_theft = LLMChain(llm=flan_ul2_llm_classify, prompt=prompt_2) - deprecated syntax
check_for_identify_theft = RunnableSequence(first=prompt_2, last=flan_ul2_llm_classify)

Creating the sequential chain.

In [ ]:
# Sequential chain:

#customer_complaint_chain = SimpleSequentialChain(chains=[get_list_of_complaints, check_for_identify_theft], verbose=True)
customer_complaint_chain = RunnableSequence(first=get_list_of_complaints, last=check_for_identify_theft)

The customer_complaint is the input being used for this exercise. You can modify this text to test the notebook. This cell block will generate the classification.

In [ ]:
# You can experiment with different values of customer complaints

customer_complaint = "I am writing you this statement to delete the following information on my credit report. \
                     The items I need deleted are listed in the report. I am a victim of identity thief, I demand that you \
                     remove these errors to correct my report immediately! I have reported this to the federal trade commission \
                     and have attached the federal trade commission affidavit. Now that I have given you the following information, \
                     please correct my credit report or I shall proceed with involving my attorney!"

chain_output = customer_complaint_chain.invoke(customer_complaint, config={'callbacks': [ConsoleCallbackHandler()]})

This cell block adds the question to the output so that it can be printed.

In [ ]:
#print(chain_output)
print("Is this customer reporting identity theft: " + chain_output)

# A production application will contain logic to trigger some action if identity theft is true

### Authors: 
 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.
 **Elena Lowery**, Data and AI Architect.
 
 Notebook updated by **Andre de Waal**, Senior Learning Content Developer.
 

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.